## Agent wydawca

In [ ]:
!pip install -q langgraph langchain langchain-openai langchain-community python-dotenv pydot


In [ ]:
import os
from typing import TypedDict, List, Literal, Dict, Any

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.types import Command, interrupt

# Model bazowy (możesz podmienić na inny, np. gpt-4o-mini / gpt-4.1-mini)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Czy mamy prawdziwy klucz do Tavily?
HAS_TAVILY = bool(os.getenv("TAVILY_API_KEY"))
print("Model OK. Tavily:", "ON" if HAS_TAVILY else "mock")


### Stan aplikacji

In [ ]:
class PublisherState(MessagesState):
    # preferencje użytkownika (uczymy się z feedbacku)
    sources: List[str] = []
    topics: List[str] = []
    # jaki tryb wybrał agent: "daily" | "fun" | "single"
    mode: Literal["daily", "fun", "single"] = "daily"
    # ostatnie surowe wyniki z narzędzia (np. listy {title, url, content})
    raw_results: List[Dict[str, Any]] = []
    # skrócone podsumowanie w Markdown
    summary_md: str = ""


### Narzędzia Tavily lub mock

In [ ]:
def get_search_tool():
    if HAS_TAVILY:
        # Prawdziwe narzędzie (zwraca listę wyników wyszukiwania)
        return TavilySearchResults(
            max_results=10,
            search_depth="advanced",  # „advanced” lub „basic”
            include_answer=False,
            include_raw_content=False,
        )
    else:
        # Lokalny mock — udaje wyniki wyszukiwania
        from langchain.tools import tool

        @tool("mock_search", return_direct=False)
        def mock_search(query: str) -> List[Dict[str, str]]:
            """MOCK: Zwraca przykładowe wyniki wyszukiwania (title, url, content)."""
            base = "https://example.com/"
            return [
                {"title": "AI & Startups Weekly", "url": base + "ai-startups", "content": "Roundup nowości AI w startupach."},
                {"title": "LangChain Updates", "url": base + "langchain", "content": "Nowe funkcje LC i ekosystem narzędzi."},
                {"title": "Funding News", "url": base + "funding", "content": "Rundy finansowania w branży technologicznej."},
                {"title": "ML Research Highlights", "url": base + "ml-paper", "content": "Przegląd ciekawych publikacji ML."},
                {"title": "Product Launches", "url": base + "launches", "content": "Nowe produkty i funkcje."},
            ]
        return mock_search

search_tool = get_search_tool()
tool_node = ToolNode([search_tool])


### Prompty systemowe

In [ ]:
AGENT_SYSTEM = """Jesteś agentem wydawcą. Twoje zadania:
1) Zrozumieć intencję użytkownika i wybrać tryb:
   - daily: dzienne podsumowanie (5–8 linków),
   - fun: jeden ciekawostkowy news + wyjaśnienie dlaczego interesujący,
   - single: podsumowanie z jednego wskazanego źródła (1–3 linki).
2) Jeśli potrzeba informacji, wywołaj narzędzie wyszukiwania.
3) Gdy masz wystarczająco materiałów, zakończ fazę planowania — NIE twórz finalnej odpowiedzi (to zrobi summarizer).

Zasady:
- Jeśli użytkownik poda źródła lub tematy, traktuj je priorytetowo.
- Jeśli pierwsze wyniki są słabe/za mało (mniej niż 5 do daily), spróbuj innego sformułowania zapytania.
- Używaj zwięzłych narzędziowych zapytań.
Zwróć decyzje poprzez tool_calls (jeśli trzeba szukać) lub od razu przejdź do podsumowania (bez tool_calls).
"""

SUMMARIZER_SYSTEM = """Sformatuj wyniki jako **Markdown**:
- dla 'daily': listy punktowane z krótkimi opisami, 5–8 pozycji
- dla 'fun': 1–2 pozycje z akcentem na ciekawostkę i „dlaczego”
- dla 'single': 1–3 pozycje z jednego źródła
Na końcu dodaj sekcję **Źródła** jako listę numerowaną z URL.
Nie wymyślaj linków — używaj tylko podanych.
"""

FEEDBACK_SYSTEM = """Jesteś menedżerem preferencji. Na podstawie wiadomości użytkownika wyodrębnij:
- sources: lista nazw/serwisów (np. 'TechCrunch', 'The Verge')
- topics: lista tematów (np. 'AI', 'fintech', 'startup funding')
Zwróć minimalny JSON: {"sources": [...], "topics": [...]}. Nic poza tym.
"""


### Logika węzłów

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

def agent_node(state: PublisherState) -> Command:
    """
    Planner: decyduje o trybie (daily/fun/single), przygotowuje i ewentualnie
    wywołuje narzędzie wyszukiwania (tool_calls). Jeśli uzna, że ma dość danych,
    przejdzie do 'summarize' bez tool_calls.
    """
    # Zbuduj wiadomości
    msgs = [SystemMessage(content=AGENT_SYSTEM)]
    if state.get("sources"):
        msgs.append(SystemMessage(content=f"Preferowane źródła: {', '.join(state['sources'])}"))
    if state.get("topics"):
        msgs.append(SystemMessage(content=f"Preferowane tematy: {', '.join(state['topics'])}"))
    msgs += state["messages"]

    # Podpowiedź dla modelu: wybierz tryb
    mode_prompt = ChatPromptTemplate.from_template(
        "Na podstawie ostatniej wiadomości użytkownika wybierz tryb: 'daily', 'fun' lub 'single'. "
        "Odpowiedz jednym słowem."
    )
    mode_result = (mode_prompt | llm | StrOutputParser()).invoke(
        {"input": ""}
    ).strip().lower()
    mode = "daily" if mode_result not in {"daily", "fun", "single"} else mode_result

    # Zwiąż narzędzia i poproś model o decyzję/ew. tool_calls
    llm_with_tools = llm.bind_tools([search_tool])

    # Podstawowe zalecenie: jeśli daily/single – najpierw szukaj; gdy fun – też spróbuj wyszukania 1–2 pozycji
    agent_decision = llm_with_tools.invoke(msgs)

    # Uaktualnij stan (zapisz nową wiadomość i wybrany tryb)
    update = {"messages": [agent_decision], "mode": mode}

    # Jeśli są tool_calls → przejdź do tools, w przeciwnym razie → summarize
    goto = "tools" if getattr(agent_decision, "tool_calls", None) else "summarize"
    return Command(update=update, goto=goto)


def summarize_node(state: PublisherState) -> PublisherState:
    """Buduje finalne podsumowanie na podstawie dostępnych wiadomości i wyników narzędzia."""
    # Wyciągnij najnowsze „surowe” wyniki z komunikatów narzędzi (dla Tavily są w AIMessage.tool_calls → ToolMessage)
    # Uproszczenie: zlecamy modelowi zebranie źródeł z całej rozmowy.
    prompt = ChatPromptTemplate.from_messages([
        ("system", SUMMARIZER_SYSTEM),
        ("user", "Tryb: {mode}\nHistoria:\n{history}\n\nUtwórz końcowe podsumowanie teraz.")
    ])
    history_text = "\n".join([f"{m.type.upper()}: {m.content}" for m in state["messages"]])
    summary = (prompt | llm | StrOutputParser()).invoke({"mode": state["mode"], "history": history_text})
    return {"summary_md": summary, "messages": [AIMessage(content=summary)]}


def feedback_node(state: PublisherState) -> Command:
    """
    Human-in-the-loop: poproś użytkownika o krótką informację zwrotną
    dotyczącą preferowanych źródeł/tematów. W notebooku skorzystamy z interrupt(),
    aby wpisać feedback ręcznie z interfejsu Jupytera.
    """
    req = {
        "action_request": {
            "action": "Podaj preferencje: źródła i/lub tematy, np. 'Źródła: TechCrunch, The Verge; Tematy: AI, startupy'. "
                      "Możesz też wpisać 'pomiń'.",
            "args": {},
        },
        "config": {
            "allow_ignore": True,
            "allow_respond": True,
            "allow_edit": False,
            "allow_accept": False,
        },
        "description": state.get("summary_md", ""),
    }
    resp = interrupt([req])[0]

    if resp["type"] == "response":
        user_text = resp["args"]
        # Zparsuj preferencje LLM-em do JSON
        parser = JsonOutputParser()
        pref_prompt = ChatPromptTemplate.from_messages([
            ("system", FEEDBACK_SYSTEM),
            ("user", "{feedback}")
        ])
        parsed = (pref_prompt | llm | parser).invoke({"feedback": user_text})
        # scal preferencje (unikalne)
        new_sources = list({*(state.get("sources", [])), *parsed.get("sources", [])})
        new_topics = list({*(state.get("topics", [])), *parsed.get("topics", [])})

        update = {"sources": new_sources, "topics": new_topics, "messages": [HumanMessage(content=user_text)]}
        goto = END
        return Command(update=update, goto=goto)

    # „ignore” lub inne → po prostu kończymy
    return Command(goto=END)


### Budowa grafu

In [ ]:
def should_continue_from_agent(state: PublisherState) -> str:
    """Jeśli ostatnia odpowiedź zawiera tool_calls → przejdź do tools; w przeciwnym razie → summarize."""
    last = state["messages"][-1]
    return "tools" if getattr(last, "tool_calls", None) else "summarize"

graph = (
    StateGraph(PublisherState)
    .add_node("agent", agent_node)
    .add_node("tools", tool_node)         # wykonanie tool_calls (np. wyszukiwania)
    .add_node("summarize", summarize_node)
    .add_node("feedback", feedback_node)
    .add_edge(START, "agent")
    .add_edge("tools", "agent")           # po narzędziach wróć do agenta (może poprosić o kolejne wyszukiwanie lub podsumować)
    .add_conditional_edges("agent", should_continue_from_agent, {"tools": "tools", "summarize": "summarize"})
    .add_edge("summarize", "feedback")    # na końcu prosimy o feedback (HITL)
    .add_edge("feedback", END)
)

app = graph.compile()
print("Graf skompilowany.")


### Uruchomienie

In [ ]:
initial: PublisherState = {
    "messages": [
        HumanMessage(content="Zrób daily briefing o AI i startupach za ostatnie 48h. Preferowane: The Verge, TechCrunch.")
    ],
    "sources": [],
    "topics": [],
    "mode": "daily",
    "raw_results": [],
    "summary_md": ""
}

final_state = app.invoke(initial)
print("\n=== PODSUMOWANIE (Markdown) ===\n")
print(final_state.get("summary_md", "brak"))


### Wizualizacja grafu

In [ ]:
from IPython.display import Image, display

png_bytes = app.get_graph().draw_png()
display(Image(png_bytes))
